# SWMAL Assignment 01
This notebook consists of two exercises, the "Intro" and the "Modules and Packages in Python" which is the required exercises for O1.

## L01 - Intro

_Below code is the code used to generate some of the figures in chapter 1._

Afterwards a list of questions will be answered.

### Setup

In [ ]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "fundamentals"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("IGNORING: Saving figure", fig_id) # SWMAL: I've disabled saving of figures
    #if tight_layout:
    #    plt.tight_layout()
    #plt.savefig(path, format='png', dpi=300)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", module="scipy", message="^internal gelsd")

print("OK")

In [ ]:
def prepare_country_stats(oecd_bli, gdp_per_capita):
    oecd_bli = oecd_bli[oecd_bli["INEQUALITY"]=="TOT"]
    oecd_bli = oecd_bli.pivot(index="Country", columns="Indicator", values="Value")
    gdp_per_capita.rename(columns={"2015": "GDP per capita"}, inplace=True)
    gdp_per_capita.set_index("Country", inplace=True)
    full_country_stats = pd.merge(left=oecd_bli, right=gdp_per_capita,
                                  left_index=True, right_index=True)
    full_country_stats.sort_values(by="GDP per capita", inplace=True)
    remove_indices = [0, 1, 6, 8, 33, 34, 35]
    keep_indices = list(set(range(36)) - set(remove_indices))
    return full_country_stats[["GDP per capita", 'Life satisfaction']].iloc[keep_indices]

print("OK")

In [ ]:
import os
datapath = os.path.join("datasets", "lifesat", "")

# NOTE: a ! prefix makes us able to run system commands..
# (command 'dir' for windows, 'ls' for Linux or Macs)
#

! dir

print("\nOK")

In [ ]:
# Code example
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn.linear_model

# Load the data
try:
    oecd_bli = pd.read_csv(datapath + "oecd_bli_2015.csv", thousands=',')
    gdp_per_capita = pd.read_csv(datapath + "gdp_per_capita.csv",thousands=',',delimiter='\t',
                             encoding='latin1', na_values="n/a")
except Exception as e:
    print(f"SWMAL NOTE: well, you need to have the 'datasets' dir in path, please unzip 'datasets.zip' and make sure that its included in the datapath='{datapath}' setting in the cell above..")
    raise e
    
# Prepare the data
country_stats = prepare_country_stats(oecd_bli, gdp_per_capita)
X = np.c_[country_stats["GDP per capita"]]
y = np.c_[country_stats["Life satisfaction"]]

# Visualize the data
country_stats.plot(kind='scatter', x="GDP per capita", y='Life satisfaction')
plt.show()

# Select a linear model
model = sklearn.linear_model.LinearRegression()

# Train the model
model.fit(X, y)

# Make a prediction for Cyprus
X_new = [[22587]]  # Cyprus' GDP per capita
y_pred = model.predict(X_new)
print(y_pred) # outputs [[ 5.96242338]]

print("OK")

In [ ]:
oecd_bli = pd.read_csv(datapath + "oecd_bli_2015.csv", thousands=',')
oecd_bli = oecd_bli[oecd_bli["INEQUALITY"]=="TOT"]
oecd_bli = oecd_bli.pivot(index="Country", columns="Indicator", values="Value")
#oecd_bli.head(2)

gdp_per_capita = pd.read_csv(datapath+"gdp_per_capita.csv", thousands=',', delimiter='\t',
                             encoding='latin1', na_values="n/a")
gdp_per_capita.rename(columns={"2015": "GDP per capita"}, inplace=True)
gdp_per_capita.set_index("Country", inplace=True)
#gdp_per_capita.head(2)

full_country_stats = pd.merge(left=oecd_bli, right=gdp_per_capita, left_index=True, right_index=True)
full_country_stats.sort_values(by="GDP per capita", inplace=True)
#full_country_stats

remove_indices = [0, 1, 6, 8, 33, 34, 35]
keep_indices = list(set(range(36)) - set(remove_indices))

sample_data = full_country_stats[["GDP per capita", 'Life satisfaction']].iloc[keep_indices]
#missing_data = full_country_stats[["GDP per capita", 'Life satisfaction']].iloc[remove_indices]

sample_data.plot(kind='scatter', x="GDP per capita", y='Life satisfaction', figsize=(5,3))
plt.axis([0, 60000, 0, 10])
position_text = {
    "Hungary": (5000, 1),
    "Korea": (18000, 1.7),
    "France": (29000, 2.4),
    "Australia": (40000, 3.0),
    "United States": (52000, 3.8),
}
for country, pos_text in position_text.items():
    pos_data_x, pos_data_y = sample_data.loc[country]
    country = "U.S." if country == "United States" else country
    plt.annotate(country, xy=(pos_data_x, pos_data_y), xytext=pos_text,
            arrowprops=dict(facecolor='black', width=0.5, shrink=0.1, headwidth=5))
    plt.plot(pos_data_x, pos_data_y, "ro")
#save_fig('money_happy_scatterplot')
plt.show()

from sklearn import linear_model
lin1 = linear_model.LinearRegression()
Xsample = np.c_[sample_data["GDP per capita"]]
ysample = np.c_[sample_data["Life satisfaction"]]
lin1.fit(Xsample, ysample)

t0 = 4.8530528
t1 = 4.91154459e-05

sample_data.plot(kind='scatter', x="GDP per capita", y='Life satisfaction', figsize=(5,3))
plt.axis([0, 60000, 0, 10])
M=np.linspace(0, 60000, 1000)
plt.plot(M, t0 + t1*M, "b")
plt.text(5000, 3.1, r"$\theta_0 = 4.85$", fontsize=14, color="b")
plt.text(5000, 2.2, r"$\theta_1 = 4.91 \times 10^{-5}$", fontsize=14, color="b")
#save_fig('best_fit_model_plot')
plt.show()

print("OK")

### Qa) The $\theta$ parameters and the $R^2$ Score
 
The $\theta$ parameters, also refered to as weights or coefficients, can be adjusted during training to optimize the performance and acurracy of a certain linear regression model. 

Extrating the $\theta_0$ and $\theta_1$ coefficients can be done by using the below python attributes:

In [ ]:
theta_0 = model.intercept_
theta_1 = model.coef_[0]

print("Theta_0 (Intercept):", theta_0)
print("Theta_1 (Coefficient):", theta_1)

 Another important variable is the $R^2$ score. 

The $R^2$ measures the goodness of fit of the model itself. The minimum and maximum values range from 0 to 1, where the below list describes each outcome:

- A value of 1: Considered a perfect fit, where the model accurately predicts the dependent variable based on the independent variables (Explains all the variance).
- A value of 0: The opposite of a perfect fit. This score indicates that the model's prediction is entirely wrong and provides no explanation.

While it is desirable to achieve a score of 1, it's important to note that such a score is highly unlikely as it's almost impossible to create a model that makes a perfect fit to real-world data.

Therefore, it is preferable to find a value close to 1, which, as explained, indicates a better fit.

In the assignment, the below calculation extracts the desirable score of 0.734.


In [ ]:
score = model.score(X, y) 
print("Score:", score)

### Qb) Using k-Nearest Neighbors

Instead of using linear regression, it will now be a k-nearest neighbour algorithm, here with k=3.

In [ ]:
sample_data

In [ ]:
# Preprocessed data
country_stats

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

# Prepare the data
X = np.c_[country_stats["GDP per capita"]]
y = np.c_[country_stats["Life satisfaction"]]

print("X.shape=",X.shape)
print("y.shape=",y.shape)

# Visualize the data
country_stats.plot(kind='scatter', x="GDP per capita", y='Life satisfaction')
plt.grid(True)
plt.show()

# Select and train a model
knn = KNeighborsRegressor(n_neighbors=3)

# Train the new model
knn.fit(X, y) 


# Predictions for Cyprus 
print("KNN prediction: ", knn.predict(X_new))  
print("Linear regression prediction: ", lin1.predict(X_new))  

# Evaluating the models 
print("KKN score: ", knn.score(X, y))
print("Linear score: ", lin1.score(X, y))





As seen in the calculations above, the KNN prediction for Cyprus is 5.77, which is a bit less than the linear prediction of 5.96. 

KNN uses the score method of the coefficient of determination (R^2), as does the linear regression model. This means they are able to be compared, since they use the same scoring methods. 


The model used here is the "Regression based on k-nearest neighbors" (URL: https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html)








### Qc) Tuning Parameter for k-Nearest Neighbors and A Sanity Check




But that not the full story. Try plotting the prediction for both models in the same graph and tune the `k_neighbor` parameter of the `KNeighborsRegressor` model.  

Choosing `k_neighbor=1` produces a nice `score=1`, that seems optimal...but is it really so good?
- No, when score=1 it means that the model fits perfectly. Though this is true in some sense, it gives an obscured view. When k_neighbor=1 the model only considers a single nearest neighboor. Then the model doesn't consider the outliers in the data, and then we get a fragile model.  

Plotting the two models in a 'Life Satisfaction-vs-GDP capita' 2D plot by creating an array in the range 0 to 60000 (USD) (the `M` matrix below) and then predict the corresponding y value will sheed some light to this. 

Now reusing the plots stubs below, try to explain why the k-nearest neighbour with `k_neighbor=1` has such a good score.
- When there's only one neighbor, it will go directly from point to point. This will give the illusion that the model fits perfectly because it only has to consider 1 neighbor. On the other hand, when the model has to consider 3 neighbors, it tries to go to all of them, which is not possible. Then it makes a mean of the data. 

Does a score=1 with `k_neighbor=1`also mean that this would be the prefered estimator for the job?
- No, when the score is 1, it suggests that there's something wrong with the model or the way that the model is used. In this scenario, when the neighbor=1, the score is 1. This suggests that the model doesn't work as intended. There could be a lot of reasons why this is the case, but in this instance, it's because the neighbor=1.

Hint here is a similar plot of a KNN for a small set of different k's:

<img src="https://itundervisning.ase.au.dk/SWMAL/L01/Figs/regression_with_knn.png"  alt="WARNING: could not get image from server." style="height:150px">

In [ ]:
sample_data.plot(kind='scatter', x="GDP per capita", y='Life satisfaction', figsize=(5,3))
plt.axis([0, 60000, 0, 10])

# create an test matrix M, with the same dimensionality as X, and in the range [0;60000] 
# and a step size of your choice
m=np.linspace(0, 60000, 1000)
M=np.empty([m.shape[0],1])
M[:,0]=m

# from this test M data, predict the y values via the lin.reg. and k-nearest models
y_pred_lin = lin1.predict(M)
y_pred_knn = knn.predict(M)   # ASSUMING the variable name 'knn' of your KNeighborsRegressor 

# use plt.plot to plot x-y into the sample_data plot..
plt.plot(m, y_pred_lin, "r")
plt.plot(m, y_pred_knn, "b")
 
# Select and train a model with neighbor 1
knn1 = KNeighborsRegressor(n_neighbors=1)
knn1.fit(X, y) 
y_pred_knn1 = knn1.predict(M)    
plt.plot(m, y_pred_knn1, "g")
print(knn1.score(X, y))

### Qd) Trying out a Neural Network

Let us then try a Neural Network on the data, using the fit-predict interface allows us to replug a new model into our existing code.

There are a number of different NN's available, let's just hook into Scikit-learns Multi-Layer Perceptron for regression, that is an 'MLPRegressor'. 

Now, the data-set for training the MLP is really not well scaled, so we need to tweak a lot of parameters in the MLP just to get it to produce some sensible output: with out preprocessing and scaling of the input data, `X`, the MLP is really a bad choice of model for the job since it so easily produces garbage output. 

Try training the `mlp` regression model below, predict the value for Cyprus, and find the `score` value for the training set...just as we did for the linear and KNN models.

Can the `MLPRegressor` score function be compared with the linear and KNN-scores?

In [ ]:
from sklearn.neural_network import MLPRegressor

# Setup MLPRegressor
mlp = MLPRegressor( hidden_layer_sizes=(10,), solver='adam', activation='relu', tol=1E-5, max_iter=100000, verbose=True)
mlp.fit(X, y.ravel())

# lets make a MLP regressor prediction and redo the plots
y_pred_mlp = mlp.predict(M) 

plt.plot(m, y_pred_lin, "r")
plt.plot(m, y_pred_knn, "b")
plt.plot(m, y_pred_mlp, "k")

print(mlp.score(X, y))

### Try training the `mlp` regression model below, predict the value for Cyprus, and find the `score` value for the training set...just as we did for the linear and KNN models.

We have trained the mlp model, which changes each time the code is run because it's trained by a neural network now.
The neural network stops the training when it doesn't improve more than tol=0.000010 for 10 consecutive epochs.


### Can the `MLPRegressor` score function be compared with the linear and KNN-scores?
They can be compared since the score function is implemented in the same way. The result is way different from the score results of the linear and knn scores, which in this case means that the mlp model is worse at the prediction since the result (R^2) is negative. This means that the predicted values are performing worse than if you where to take the average value of the data as a prediction. (Læs lige op på det her?)


## L01 - Modules and Packages in Python

#### Qa) Load and test the `libitmal` module

Here, we import the libitmal module from utils, which was given in the assignment. We print its directory and its content, and lastly call its TestAll() method.

In [ ]:
import numpy as np
from libitmal import utils as itmalutils

print(dir(itmalutils))
print(itmalutils.__file__)

itmalutils.TestAll()

#### Qb) Create your own module, with some functions, and test it

Here we will import our own module named `extraUtils`. The module is imported from `coolModule`, where its functions now becomes available.

In this example, the `printNogetSejt()` is used to print a predfined text along with a given parameter string

In [ ]:
from extraUtils import coolModule as module

module.printNogetSejt("Thomas")

print(module.__file__)

#### Qc) How do you 'recompile' a module?

When changing the module code, Jupyter will keep running on the old module. How do you force the Jupyter notebook to re-load the module changes? 

You can force a reload of a module by using the `reload()` function from the `importlib` module.

In [ ]:
from importlib import reload
from extraUtils import coolModule as module   

reload(module)

It is also possible to automatically reload modules before executing code by using magic commands such as `%load_ext` and `%autoreload`

Source: https://saturncloud.io/blog/jupyter-notebook-reload-module-a-comprehensive-guide/

In [ ]:
%load_ext autoreload
%autoreload 2

### Classes in Python

#### Qe) Extend the class with some public and private functions and member variables 


Below, the class MyClass has been defined with both a private and a public function. 

Private funtions are defined by the two prefixed underscores (__), while member variables are defined by the "self." annotation. This self in python classes represents an instance of the class, therefor making it possible to access methods and attributes inside the class itself. 

In [ ]:
class MyClass:
    # Public function 
    def myfun():
        # Member variable 
        self.myvar = "blah"
        print(f"This is a message inside the class, myvar={self.myvar}.")
        
        # Private function 
    def __private_function(self):
        self.asdf = "asdf"
        print(f"This is a private function, {self.asdf}.")
        
    def something():
        print("Something")
        

Now an instance of the class is created. 

In [ ]:
myObjectx = MyClass()

When trying to call a function with the self parameter missing, it throws an error, because the program is passing it an instance of a class, but it doesn't take one. 

In [ ]:
myObjectx.something()

When trying to call the private function, it fails because there is no function available called __private_function() 

In [ ]:
myObjectx.__private_function() 

If one wants to use a method with a self parameter, one needs to call it with the class directly and not an instance of the class. 

In [ ]:
MyClass.something() 

#### Qf) Extend the class with a Constructor


Below is shown how to define a constructor in a Python class, namely the \_\_init\_\_ function. 
This function will be called every time a class is instantiated. 



In [ ]:
class myClass:
    def __init__(self):
        self.att1 = 2
        self.att2 = 3
        self.att3 = 4
        

As well as a constructor, it is also possible to make a destructor in a Python class. Below it is shown how this \_\_del\_\_ function is defined in a class. 


The destructor function is not necessary in Python, since the language has garbage collection which handles the memory management automatically. 
So a detructor is not strictly necessary, except if you want to handle the memory management yourself.




In [ ]:
class myClass: 
    def __del__(self):
        print("The object has been deleted!")

By calling the destructor on an instance of the class, it will delete the object. 

In [ ]:
myobejctx = myClass() 
del myobejctx

#### Qg) Extend the class with a to-string function


... 
MISSING 